In [10]:
import numpy as np
import pandas as pd
import csv
from sklearn.cluster import MeanShift

In [5]:
with open('checkins.dat') as dat_file, open('checkins.csv', 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    for line in dat_file:
        row = [field.strip() for field in line.split('|')]
        if len(row) == 6 and row[3] and row[4]:
            csv_writer.writerow(row)

In [9]:
data = pd.read_csv("checkins.csv", sep=',', header=0)
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18


In [12]:
data.columns

Index(['id', 'user_id', 'venue_id', 'latitude', 'longitude', 'created_at'], dtype='object')

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание:на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.



In [21]:
# initialize training set
X = np.array(data.loc[:100000 - 1, ['latitude', 'longitude']])
X.shape

(100000, 2)

In [22]:
# initialize clustering model
model = MeanShift(bandwidth=0.1)

model.fit(X)

MeanShift(bandwidth=0.1)

In [23]:
model.__dict__

{'bandwidth': 0.1,
 'seeds': None,
 'bin_seeding': False,
 'cluster_all': True,
 'min_bin_freq': 1,
 'n_jobs': None,
 'max_iter': 300,
 'n_features_in_': 2,
 'n_iter_': 22,
 'cluster_centers_': array([[  40.7177164 ,  -73.99183542],
        [  33.44943805, -112.00213969],
        [  33.44638027, -111.90188756],
        ...,
        [ -37.8229826 ,  145.1811902 ],
        [ -41.2924945 ,  174.7732353 ],
        [ -45.0311622 ,  168.6626435 ]]),
 'labels_': array([ 5,  7, 30, ..., 25, 19,  4], dtype=int64)}

In [30]:
df = pd.DataFrame(data=model.cluster_centers_)
df.to_csv("cluster_centers.csv", index=False, header=False)

3231

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

Importing counter to count number of items in a cluster

In [45]:
from collections import Counter

c = Counter(model.labels_)

large_cluster_indices = [key for key in c.keys() if dict(c)[key] > 15]

In [46]:
len(large_cluster_indices)

592

In [47]:
df = pd.DataFrame(data=model.cluster_centers_[large_cluster_indices])
df.to_csv("large_cluster_centers.csv", index=False, header=False)

При желании увидеть получившиеся результаты на карте можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт mapcustomizer.com имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

In [51]:
offices = pd.read_csv('offices.txt', names=['x', 'y'], sep=',')

offices

,x,y
0,33.751277,-118.188740
1,25.867736,-80.324116
2,51.503016,-0.075479
3,52.378894,4.885084
4,39.366487,117.036146
5,-33.868457,151.205134


Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [53]:
def distance_to_nearest_office(x, y):
    
    office_x = np.array(offices.x)
    office_y = np.array(offices.y)
    
    distances = np.sqrt((office_x - x) ** 2 + (office_y - y) ** 2)
    
    return distances.min()

In [61]:
df.columns = ['x', 'y']

df

,x,y
0,38.886165,-77.048783
1,33.766636,-84.393289
2,45.523483,-122.676280
3,40.759600,-111.896078
4,33.449438,-112.002140
...,...,...
587,-33.050462,-71.616452
588,-8.664363,115.211340
589,50.111512,8.680506
590,42.016667,-94.376667


In [64]:
df['distance'] = df.apply(lambda entry: distance_to_nearest_office(entry['x'], entry['y']), axis=1)

df

,x,y,distance
0,38.886165,-77.048783,13.424131
1,33.766636,-84.393289,8.885426
2,45.523483,-122.676280,12.598526
3,40.759600,-111.896078,9.418821
4,33.449438,-112.002140,6.193959
...,...,...,...
587,-33.050462,-71.616452,59.558186
588,-8.664363,115.211340,43.940864
589,50.111512,8.680506,4.421114
590,42.016667,-94.376667,21.407058


In [78]:
nearest_clusters = np.argsort(np.array(df['distance']))

nearest_clusters[:20]

array([139, 270, 355, 118,  18, 165, 140, 112, 184,  93, 124, 248, 278,
       135, 106,  11,  15, 134,  52,  16], dtype=int64)

In [83]:
nearest = df.loc[139, :]
nearest

x           -33.860630
y           151.204776
distance      0.007835
Name: 139, dtype: float64

In [84]:
with open('ans.txt', 'w') as f:
    f.write(f"{nearest.x} {nearest.y}")